In [57]:
import pandas as pd

In [58]:
# Read the data into a Pandas DataFrame
crowdfunding_data = pd.ExcelFile("crowdfunding.xlsx")
# Get the sheet names.
crowdfunding_data.sheet_names

['crowdfunding_info', 'contact_info']

In [ ]:
# Get the crowdfunding_info from the crowdfunding_info worksheet. 
crowdfunding_info_df = pd.read_excel(crowdfunding_data, sheet_name='crowdfunding_info')
crowdfunding_info_df.head()


In [ ]:
# Get the contact_info from the contact_info worksheet.
# Increase the width of the column.
pd.set_option('max_colwidth', 400)
contact_info_df = pd.read_excel(crowdfunding_data, sheet_name='contact_info', header=3)
contact_info_df.head()


In [ ]:
# Get a brief summary of the crowdfunding_info DataFrame. 
crowdfunding_info_df.info()

In [ ]:
# Get a brief summary of the contact_info DataFrame. 
contact_info_df.info()

In [ ]:
# Get the unique values in the category & sub-category column. 
crowdfunding_info_df["category & sub-category"].unique()

In [ ]:
# Assign the category and subcategory values to category and subcategory columns.
crowdfunding_info_df[["category","subcategory"]] = crowdfunding_info_df["category & sub-category"].str.split('/', n=1, expand=True)
crowdfunding_info_df.head(10)


In [ ]:
# Get the distinct values in the category and subcategory columns.
print(crowdfunding_info_df["category"].nunique())
print(crowdfunding_info_df["subcategory"].nunique())

In [ ]:
# Get the unique categories and subcategories in separate lists.
categories = crowdfunding_info_df["category"].unique()
subcategories = crowdfunding_info_df["subcategory"].unique()
print(categories)
print()
print(subcategories)

In [ ]:
# Import the Numpy dependency
import numpy as np
# Create numpy arrays that have 1-10 for the category_ids and 1-25 for the subcategory_ids.
category_ids = np.arange(1, 10)
subcategory_ids = np.arange(1, 25)

In [ ]:
# Use a list comprehension to add "cat0" to each category_id. 
cat_ids = ["cat0" + str(cat_id) for cat_id in category_ids]
print(cat_ids)


In [ ]:
# Use a list comprehension to add "scat0" to each subcategory_id. 
scat_ids = ["scat0" + str(scat_id) for scat_id in subcategory_ids ]
print(scat_ids)

In [ ]:
# Create a category DataFrame with the cat_ids array as the category_id and categories list as the category name.
category_df = pd.DataFrame({
    "category_id": cat_ids,
    "category" : categories})
# Create a subcategory DataFrame with the scat_ids array as the subcategory_id and subcategories list as the subcategory name.
subcategory_df = pd.DataFrame({
    "subcategory_id": scat_ids,
    "subcategory" : subcategories})


In [ ]:
category_df.to_csv("category_df.csv", index=False)
subcategory_df.to_csv("subcategory_df.csv", index=False)


In [ ]:
# Create a copy of the crowdfunding_info_df DataFrame name campaign_df.
campaign_df = crowdfunding_info_df.copy()
campaign_df.head()

In [ ]:
# Rename the blurb, launched_at, and deadline columns.
campaign_df = campaign_df.rename(columns={'blurb': 'description', 'launched_at': 'launched_date', 'deadline': 'end_date'})


In [ ]:
# Convert the goal and pledged columns to a `float` data type.
campaign_df[["goal","pledged"]] = campaign_df[["goal","pledged"]].astype(float)
campaign_df.head()

In [ ]:
# Format the launched_date and end_date columns to datetime format.
from datetime import datetime as dt
campaign_df["launched_date"] = pd.to_datetime(campaign_df["launched_date"], unit='s').dt.strftime('%Y-%m-%d') 
campaign_df["end_date"] = pd.to_datetime(campaign_df["end_date"], unit='s').dt.strftime('%Y-%m-%d')
campaign_df.head()

In [ ]:
# Merge the campaign_df with the category_df on the "category" column and 
# the subcategory_df on the "subcategory" column.
campaign_merged_df = campaign_df.merge(category_df, on='category', how='left').merge(subcategory_df, on='subcategory', how='left')
campaign_merged_df.tail(10)


In [ ]:
# Drop unwanted columns.
campaign_cleaned = campaign_merged_df.drop(['staff_pick', 'spotlight', 'category & sub-category','category', 'subcategory'], axis=1)
campaign_cleaned.head()


In [ ]:
# Show the contact_info_df DataFrame (get attribute error)
contact_info_df.head()

In [ ]:
# Retrieve the data from the "contact_info" column.
contact_info_list = contact_info_df.contact_info.to_list()
contact_info_list

In [ ]:
# Pop out the unique identification number using list slicing. 
print(contact_info_list[0][:15])
print(contact_info_list[0][15:19])

In [ ]:
# Pop out the unique identification number using list comprehension and slicing. 
print([x[15:19] for x in contact_info_list])

In [ ]:
# Extract the four-digit contact ID number and add it to a new column in the `campaign_cleaned` DataFrame.
campaign_cleaned['contact_id'] = [x[15:19] for x in contact_info_df['contact_info'].values]
campaign_cleaned.head()


In [ ]:
# Convert the "contact_id" column to an int64 data type.
campaign_cleaned['contact_id'] = pd.to_numeric(campaign_cleaned['contact_id'])
campaign_cleaned.info()

In [ ]:
# Place the columns in the following order:
# "cf_id", "contact_id", "company_name", "description", "goal", "pledged", "outcome", "backers_count", "country", "currency", "launched_date", "end_date", "category_id","subcategory_id"
campaign_cleaned = campaign_cleaned[['cf_id','contact_id',
                                           'company_name', 'description', 
                                           'goal', 'pledged', 'outcome', 
                                           'backers_count', 'country', 'currency', 
                                           'launched_date', 'end_date', 'category_id', 
                                           'subcategory_id']]
campaign_cleaned



In [ ]:
# Export the DataFrame as a CSV named campaign.csv, with encoding='utf8' and without the index.
campaign_cleaned.to_csv("campaign.csv", encoding='utf8', index=False)


In [ ]:
# Extract the four-digit contact ID number and add it to a new column in the contact_info DataFrame.
contact_info_df["contact_id"] = [x[15:19] for x in contact_info_df["contact_info"].values]
contact_info_df.head()


In [ ]:
# Get the data from the first row and convert it to a dictionary.
import json
first_row = contact_info_df.iloc[0]['contact_info']
converted_data = json.loads(first_row)
for k, v in converted_data.items():
    print(k,v)

In [ ]:
# Iterate through the contact_info_df and convert each row to a dictionary.
dict_values = []
for i, row in contact_info_df.iterrows():
    # Get the data first item in each row.
    data = row['contact_info']
    converted_data = json.loads(data)
    # Iterate through each dictionary (row) and get the values for each row using list comprehension.
    row_values = [v for k, v in converted_data.items()]
    # Append the list of values for each row to a new list. 
    dict_values.append(row_values)

# Print out the list of values for each row.
print(dict_values)

In [ ]:
# Create a contact_info DataFrame and add each list of values, i.e., each row to the 'contact_id', 'name', 'email' columns.
contacts_df = pd.DataFrame(dict_values, columns=['contact_id', 'name', 'email'])
contacts_df.head()

In [ ]:
# Assign the first name and last name to separate columns.
contacts_df[["first_name","last_name"]]=contacts_df["name"].str.split(" ", n=1, expand=True)
contacts_df.head()


In [ ]:
# Drop the name column. 
contacts_df_clean = contacts_df.drop(['name'], axis=1)
contacts_df_clean.head(10)


In [ ]:
# Organize new columns
new_columns=["contact_id", "first_name", "last_name", "email"]
contacts_df_clean=contacts_df_clean[new_columns]
contacts_df_clean.head()


In [ ]:
# Save new csv file
contacts_df_clean.to_csv("contacts.csv", encoding='utf8', index=False)